In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import re
import pandas as pd

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

# Usersdf EDA and Cleaning

In [2]:
usersdf = pd.DataFrame.from_csv('finalproject-users.csv')

In [3]:
print len(usersdf)
usersdf.head(200)

11323


,location_state,user_id,climbs_trad,gender,age,climbs_aid,climbs_ice,climbs_boulder,user_link,climbs_mixed,location_place,climbs_sport,member_date,user_name,lead_diff_sport,boulder_diff,follow_diff_sport,lead_diff_ice,lead_diff_trad,follow_diff_ice,follow_diff_trad,follow_diff_mixed,lead_diff_mixed,follow_diff_aid,lead_diff_aid
10101,WA,10101,0,Male,37,0,0,0,https://www.mountainproject.com/u/michael-koma...,0,Seattle,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",michael-komarnitsky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10102,undef,10102,1,Male,0,1,1,0,https://www.mountainproject.com/u/ben-mottinge...,1,undef,1,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",ben-mottinger,5.10d,NaN,5.11c,WI4,5.10a,WI6,5.11a,M8,M5,C1,C3
10106,NM,10106,1,Male,35,1,0,1,https://www.mountainproject.com/u/patrick-vern...,0,Albuquerque,1,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",patrick-vernon,5.13a,V8,5.13a,NaN,5.12d,NaN,5.12d,NaN,NaN,C2,C2
10107,undef,10107,0,undef,0,0,0,0,https://www.mountainproject.com/u/matt-bauman/...,0,undef,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",matt-bauman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10108,CO,10108,0,Male,36,0,0,0,https://www.mountainproject.com/u/quinn-steven...,0,Denver,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",quinn-stevens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10109,WI,10109,0,Male,0,0,0,0,https://www.mountainproject.com/u/ramin-jamshi...,0,Wauwatosa,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",ramin-jamshidi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10112,undef,10112,0,undef,0,0,0,0,https://www.mountainproject.com/u/nate-weitzel...,0,undef,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",nate-weitzel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10113,undef,10113,0,undef,0,0,0,0,https://www.mountainproject.com/u/walt-wehner/...,0,undef,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",walt-wehner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10116,AZ,10116,1,Male,34,0,0,1,https://www.mountainproject.com/u/peter-franze...,0,Phoenix,1,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",peter-franzen,5.13a,V9,5.13a,NaN,5.11c,NaN,5.12a,NaN,NaN,NaN,NaN
10117,undef,10117,0,undef,0,0,0,0,https://www.mountainproject.com/u/andrew-wellm...,0,undef,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",andrew-wellman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
usersdf.describe()

,user_id,climbs_trad,age,climbs_aid,climbs_ice,climbs_boulder,climbs_mixed,climbs_sport
count,1.132300e+04,11323.000000,11323.000000,11323.000000,11323.000000,11323.000000,11323.000000,11323.000000
mean,1.019186e+08,0.481939,15.086726,0.081162,0.129383,0.366864,0.055816,0.542436
std,2.403621e+07,0.499696,17.085963,0.273096,0.335638,0.481970,0.229575,0.498218
min,1.010100e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.063474e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.072737e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.082118e+08,1.000000,30.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.113856e+08,1.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# fix age such that where there was no age, it is set to NaN and not 0
usersdf.age.replace(0,np.nan, inplace=1)

In [6]:
print "Total User Profiles Scraped: ", len(usersdf)
print "Undeclared Gender: ", len(usersdf[usersdf.gender == 'undef'])
print "Declared Men: ", len(usersdf[usersdf.gender == 'Male'])
print "Declared Women: ", len(usersdf[usersdf.gender == 'Female'])

Total User Profiles Scraped:  11323
Undeclared Gender:  4309
Declared Men:  6222
Declared Women:  792


^^^^ YIKES!

points to note:
- women might be reporting gender at a lower rate
- it is generally well known that the climbing community is predominantly male, though the usual anecdotal estimate is about a 30-70 ratio...

In [7]:
# convert male,female to binary, numeric and add numpy.nan
usersdf.gender.replace(['Male','Female','undef'],[0,1,np.nan], inplace=1)

In [8]:
# check/replace if any 'None' or 'undef' strings
usersdf.replace(['None','undef'], [np.nan,np.nan], inplace=1)


In [9]:
usersdf.head(10)

,location_state,user_id,climbs_trad,gender,age,climbs_aid,climbs_ice,climbs_boulder,user_link,climbs_mixed,location_place,climbs_sport,member_date,user_name,lead_diff_sport,boulder_diff,follow_diff_sport,lead_diff_ice,lead_diff_trad,follow_diff_ice,follow_diff_trad,follow_diff_mixed,lead_diff_mixed,follow_diff_aid,lead_diff_aid
10101,WA,10101,0,0,37,0,0,0,https://www.mountainproject.com/u/michael-koma...,0,Seattle,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",michael-komarnitsky,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10102,NaN,10102,1,0,NaN,1,1,0,https://www.mountainproject.com/u/ben-mottinge...,1,NaN,1,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",ben-mottinger,5.10d,NaN,5.11c,WI4,5.10a,WI6,5.11a,M8,M5,C1,C3
10106,NM,10106,1,0,35,1,0,1,https://www.mountainproject.com/u/patrick-vern...,0,Albuquerque,1,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",patrick-vernon,5.13a,V8,5.13a,NaN,5.12d,NaN,5.12d,NaN,NaN,C2,C2
10107,NaN,10107,0,NaN,NaN,0,0,0,https://www.mountainproject.com/u/matt-bauman/...,0,NaN,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",matt-bauman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10108,CO,10108,0,0,36,0,0,0,https://www.mountainproject.com/u/quinn-steven...,0,Denver,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",quinn-stevens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10109,WI,10109,0,0,NaN,0,0,0,https://www.mountainproject.com/u/ramin-jamshi...,0,Wauwatosa,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",ramin-jamshidi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10112,NaN,10112,0,NaN,NaN,0,0,0,https://www.mountainproject.com/u/nate-weitzel...,0,NaN,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",nate-weitzel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10113,NaN,10113,0,NaN,NaN,0,0,0,https://www.mountainproject.com/u/walt-wehner/...,0,NaN,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",walt-wehner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10116,AZ,10116,1,0,34,0,0,1,https://www.mountainproject.com/u/peter-franze...,0,Phoenix,1,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",peter-franzen,5.13a,V9,5.13a,NaN,5.11c,NaN,5.12a,NaN,NaN,NaN,NaN
10117,NaN,10117,0,NaN,NaN,0,0,0,https://www.mountainproject.com/u/andrew-wellm...,0,NaN,0,"(2001, 1, 1, 0, 0, 0, 0, 1, -1)",andrew-wellman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
usersdf.describe()

,user_id,climbs_trad,gender,age,climbs_aid,climbs_ice,climbs_boulder,climbs_mixed,climbs_sport
count,1.132300e+04,11323.000000,7014.000000,5291.000000,11323.000000,11323.000000,11323.000000,11323.000000,11323.000000
mean,1.019186e+08,0.481939,0.112917,32.286335,0.081162,0.129383,0.366864,0.055816,0.542436
std,2.403621e+07,0.499696,0.316514,8.330090,0.273096,0.335638,0.481970,0.229575,0.498218
min,1.010100e+04,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.063474e+08,0.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.072737e+08,0.000000,0.000000,31.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.082118e+08,1.000000,0.000000,36.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.113856e+08,1.000000,1.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
girls_for_ian = usersdf[(usersdf.lead_diff_sport_number > 12) &
        (usersdf.gender == 0)]

len(girls_for_ian)

1930

## Climbing Grade Conversion

In order to be able to analyze the climbing grades, we need to map them to some sort of numeric scale. Functions for doing so below!

In [12]:
# rock letter to value conversion
def letter_to_num(letter):
    if letter == 'a':
        return 0.
    if letter == 'b':
        return .5
    if letter == 'c':
        return 1.
    if letter == 'd':
        return 1.5

# double digit +,- conversion
def dd_sign_to_num(sign):
    if sign == '-':
        return -.2
    elif sign == '+':
        return .2

# single digit +,- conversion
def sd_sign_to_num(sign):
    if sign == '-':
        return -.3
    elif sign == '+':
        return .3
    
def rock_grade_convert(string):
    #def rock_grade_conversion(gradestr):
    if isinstance(string, str):
        if string == '0':
            return np.nan
        number = 0
        searchobj = re.search(r'5.(\d)(\d)?([a-d])?([+,-])?', string)
    
        # if double digit grade
        if searchobj.group(2):
            number = 10 + int(searchobj.group(2)) * 2

            # check for letter 
            if searchobj.group(3):
                number += letter_to_num(searchobj.group(3))

            if searchobj.group(4):
                number += dd_sign_to_num(searchobj.group(4))

        # if single digit grade
        elif searchobj.group(1):
            number = int(searchobj.group(1))

            if searchobj.group(4):
                number += sd_sign_to_num(searchobj.group(4))

        return number
    else:
        return np.nan

In [13]:
usersdf.columns

Index([u'location_state', u'user_id', u'climbs_trad', u'gender', u'age', u'climbs_aid', u'climbs_ice', u'climbs_boulder', u'user_link', u'climbs_mixed', u'location_place', u'climbs_sport', u'member_date', u'user_name', u'lead_diff_sport', u'boulder_diff', u'follow_diff_sport', u'lead_diff_ice', u'lead_diff_trad', u'follow_diff_ice', u'follow_diff_trad', u'follow_diff_mixed', u'lead_diff_mixed', u'follow_diff_aid', u'lead_diff_aid'], dtype='object')

In [14]:
for column in [u'lead_diff_sport',u'lead_diff_trad',u'follow_diff_trad', u'follow_diff_sport']:
    usersdf[column + "_number"] = usersdf[column].apply(rock_grade_convert)

ERROR: No traceback has been produced, nothing to debug.


In [15]:
usersdf.describe()

,user_id,climbs_trad,gender,age,climbs_aid,climbs_ice,climbs_boulder,climbs_mixed,climbs_sport,lead_diff_sport_number,lead_diff_trad_number,follow_diff_trad_number,follow_diff_sport_number
count,1.132300e+04,11323.000000,7014.000000,5291.000000,11323.000000,11323.000000,11323.000000,11323.000000,11323.000000,5992.000000,5249.000000,5216.000000,5897.000000
mean,1.019186e+08,0.481939,0.112917,32.286335,0.081162,0.129383,0.366864,0.055816,0.542436,11.370577,9.217375,11.300613,12.384518
std,2.403621e+07,0.499696,0.316514,8.330090,0.273096,0.335638,0.481970,0.229575,0.498218,2.863089,2.805223,2.701004,2.884386
min,1.010100e+04,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.063474e+08,0.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,8.000000,10.000000,11.500000
50%,1.072737e+08,0.000000,0.000000,31.000000,0.000000,0.000000,0.000000,0.000000,1.000000,11.500000,9.000000,11.500000,12.500000
75%,1.082118e+08,1.000000,0.000000,36.000000,0.000000,0.000000,1.000000,0.000000,1.000000,13.500000,11.000000,13.000000,14.000000
max,1.113856e+08,1.000000,1.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,19.500000,19.500000,19.500000,19.500000


# ClimbUserDF EDA and Cleaning

In [ ]:
#ucdf = pd.DataFrame.from_csv('finalproject-sorted_climbuserdf.csv')


In [ ]:
ucdf.describe()

In [ ]:
#climbsdf = pd.DataFrame.from_csv('finalproject-sorted_climbs.csv')